# Hands on with Machine Learning: Application to space physics

_Gautier Nguyen_

## Introduction

You've seen this week, the supersonic solar wind is slowed down on its arrival in the Earth vicinity separating the near-Earth environment into three distinct regions: the magnetosphere, a hot and tenuous plasma at rest, the magnetosheath, a denser, faster and colder plasma, and the solar wind itself. Each of these regions being separated by two boundaries: the bow shock and the magnetopause. The study of these boundaries is the very first step of the study of the coupling that exist between Earth and the solar wind.

<div style="text-align: center;">
<img src="schema_context.png" width="50%">
</div>



From an observational point of view, these boundaries can be studied from a statistical point of view with the in-situ data measured by spacecrafts that come pass them. Nevertheless, this identification can be time consuming and time consuming because of the variability from the criteria used from an observer to another and the variability of the data depending on the orbit type of the spacecraft.


In this Notebook, we will elaborate an automatic detection method of the three near earth regions and use it to elaborate our own Magnetopause crossing catalog (that could eventually be adapted to the bow shock).

We will then use these catalogs to build a data-driven magnetopause model from in-situ data.


## Get the data

For the first part of this session, we will elaborate a model that classifies in-situ data into one of the three surrounding near-earth region: Magnetosphere (MSP), Magnetosheath (MSH) and solar wind (SW). For this, we will use the data provided by the mission THEMIS. This mission consists of five micro satellites (A, B, C, D and E) that all have elliptical near-equatorial orbit with various apogees as shown below:

<div style="text-align: center;">
<img src="stage-5_2010_thumb.jpg" width="20%">
</div>
Here, we will elaborate our method with the data provided by THEMIS B, the spacecraft with the largest apogee
and apply it to the data provided by THEMIS C, the spacecraft with the second largest apogee


In [ ]:
#import cells
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import datetime
from matplotlib.cm import get_cmap
import matplotlib.colorbar as colorbar
from matplotlib.colors import LogNorm
import matplotlib.dates as mdates
import seaborn as sns


import event as evt
import models

from scipy.signal import medfilt
%matplotlib inline

In [ ]:
def plotData(event, data, spectro=None, delta = datetime.timedelta(hours=3), pred=None, color='k', theo_pred=None):
    # for a given event (a specific object defined with its beginning and ending date), plot the associated data
    
    start = event.begin-delta
    end = event.end+delta
    
    n_plots = 3
    if pred is not None:
        n_plots +=1
    if spectro is not None:
        n_plots +=1
    
    fig = plt.figure(figsize=(10,15))

    ax1 = plt.subplot2grid((n_plots,9), (0,0), colspan=9)
    ax1.plot(data[start:end]['Np'], color='black')
    ax1.set_yscale('log')
    ax1.set_ylabel('$N_{p} (cm^{-3})$', fontsize=15)
    ax1.set_ylim((0.1, 70))
    

    ax2 = plt.subplot2grid((n_plots,9), (1,0), colspan=9, sharex=ax1)
    ax2.plot(data[start:end]['Bx'], color='green')
    ax2.plot(data[start:end]['By'], color='blue')
    ax2.plot(data[start:end]['Bz'], color='red')
    ax2.set_ylim(-60, 60)
    ax2.set_ylabel('B (nT)', fontsize=15)
    ax2.legend(['Bx', 'By', 'Bz '], fontsize=12,loc='center left', bbox_to_anchor=(1, 0.5))


    ax3 = plt.subplot2grid((n_plots,9), (2,0), colspan=9, sharex=ax1)
    ax3.plot(data[start:end]['Vx'], color='green')
    ax3.plot(data[start:end]['Vy'], color='blue')
    ax3.plot(data[start:end]['Vz'], color='red')
    ax3.set_ylim(-500, 300)
    ax3.set_ylabel('V(km/s)', fontsize=15)
    ax3.legend(['Vx', 'Vy', 'Vz '], fontsize=12,loc='center left', bbox_to_anchor=(1, 0.5))
    
    axarr = [ax1, ax2, ax3]
    
    
    if spectro is not None:
        ax4 = plt.subplot2grid((n_plots,9), (3,0), colspan=9, sharex=ax1)
        seum = spectro[start:end]
        t_data = seum.index
        data_tmp = np.flip(seum.values.T, 0)
        data_tmp[data_tmp < 0] = 0
        xx, yy = np.meshgrid(t_data, np.arange(0, len(seum.columns)))
        im = ax4.pcolormesh(xx, yy, data_tmp, norm=colors.LogNorm(vmin=1, vmax=1e9), cmap='nipy_spectral')
        ax4.set_ylabel('Ions energy (eV)', fontsize=15)
        ax4.set_yticks([1, 10, 18, 26])
        ax4.set_yticklabels([10, 100, 1000, 10000], fontsize=12)
        cbaxes = fig.add_axes([0.92, 0.13, 0.03, 0.15]) 
        cbar = plt.colorbar(im, cax = cbaxes)
        cbar.ax.tick_params(labelsize=12) 
        axarr.append(ax4)
    if pred is not None:
        ax8 = plt.subplot2grid((n_plots,9), (n_plots-1,0), colspan=9, sharex=ax1)
        if theo_pred is not None:
            ax8.plot(theo_pred+0.2, color='b', alpha=0.5)
        ax8.plot(pred, color=color)
        ax8.set_ylim(-0.01, 2.4)
        axarr.append(ax8)
        ax8.legend(['Label', 'Prediction'], fontsize=12)
    
    for ax in axarr:
        ax.yaxis.set_tick_params(labelsize=12)
        ax.xaxis.set_tick_params(labelsize=12)
    
        ax.axvline(event.begin, ls='dashed', color='k')
        ax.axvline(event.end, ls='dashed', color='k')
        
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:'+'%M'))
    return fig, axarr

# Classification of the three near-earth regions: THEMIS data

## Getting familiar with THEMIS data

At first, let's have a look at how the three regions look like over THEMIS B data. The data used in this part can be found  [here](https://drive.google.com/open?id=1meBpFRaXWzDYWulMXwiCRLJpkA76cqVm). Download the data file and extract it in the NSR2019 folder.

__Plot THEMIS B data for several time intervals, what can you firstly conclude about the differences between the three regions ?__

In [ ]:
#load THEMIS B data
data_themis_B = pd.read_pickle('data_themisB.pkl')
spectro_themis_B = pd.read_pickle('spectro_themisB.pkl')


y = pd.read_csv('themisB_label.csv', index_col=0, header=None, squeeze=True)
y.index = pd.to_datetime(y.index)

In [ ]:
start = datetime.datetime(2008,5,12, 8)
end =  datetime.datetime(2008,5,12, 20)
event = evt.Event(start, end)


plotData(event, data_themis_B, spectro=spectro_themis_B, delta=datetime.timedelta(hours=0))

In the rest of the study, we have discarded part of the data that corresponds to the moments where the spacecraft is in the night side part of the magnetopause, close from the Earth Dipole or too far away in the solar wind. The remnant part of the data has been labelled depending on the corresonding near-earth region data through the variable _y_. 

0 corresponds to the MSP, 1 to the MSH and 2 to the SW.

In the section 2.2 and 2.3, we will use the variable _X_ that corresponds to the part of the data that have been labelled.

In [ ]:
X = data_themis_B[data_themis_B.index.isin(y.index)]

__From your first a priori from the data you observed, what would first mere classification would you suggest ?__

## A simple first method

From the differences you hve noticed between the three near-earth regions, you could have thought of a basic first classification method. In this section, we will implement this method and evaluate this performance.

__Using _hist2d_, represent the 2d histogram of the two most useful features you have identified to differentiate the three regions__

In [ ]:
# 2D histogram of two features of the dataset

plt.figure(figsize=(15, 10))
plt.hist2d(X['Np'], X['Vx'], cmap='jet',bins = [np.arange(0, 15, 0.1),np.arange(-600, 150, 1)], norm=LogNorm())
plt.colorbar()


plt.plot([0, 3], [-100, 150], color='r')
plt.plot([0, 5], [-100, -600], color='r')

__By setting thresholds on these two parameters classify the different regions for THEMIS B data__

In [ ]:
y_pred = pd.Series(index = y.index, data=1)

y_pred[X['Vx']>250/3*X['Np']-100] = 0
y_pred[X['Vx']<-100*X['Np']-100] = 2

It is the necessary to evaluate the quality of our prediction to ensure the reliability of our model. For this, we will compare the prediction $y_{pred}$ to the true label $y$. This comparison can be summarized by two variables for each classes: 
- The Precision $\frac{TP}{TP+FN}$ where $TP$ is the number of correctly predicted points of a given class (e.g MSH points predicted as MSH) and FN are points of a given class that have been mispredicted (e.g MSH pints labelled as MSP)

- The Recall $\frac{TP}{TP+FP}$ where $FP$ is the number of points from other classes that have been predicted as belonging to a given class (e.g SW points that have been predicted as MSH)


__Using the functions $recall\_score$ and $precision\_score$ of $sklearn$, quantify the performance of this classification method, what can you conclude ?__

In [ ]:
from sklearn.metrics import precision_score, recall_score
print(recall_score(y, y_pred, average=None))
print(precision_score(y, y_pred, average=None))


You have here completed the automatic identification attempt realized by _Jelinek et al. (2012)_

## Smarter methods: Machine Learning

You understood it, all the point about classification is about establishing accurate enough boundaries between the different classes (here the three near-earth regions) in order to have a recall and a precision as high as possible.
This is the purpose of Machine-Learning algorithms.

In this section, we will try several typical supervised machine learning algorithms to figure out which one would be the best to classify the three near earth regions.

To figure this out, each model you will implement will be fit with 2/3 of the dataset that will constitute the __training set__ and will be evaluated on the remnant dataset that will constitute the __test set__. To ensure that the obtained performances does not depend on the split that will be made between training and test set, we will make this process for 3 different splits (realizing what we call __cross-validation__) and we will then provide an average precision and recall for each classes.

Feel free to try the various existing algorithms and eventually fine tune them to increase your classification performances:
- Logistic Regression
- Decision Tree
- Random Forest
- Gradient Boosting
- Support Vector Machine

The complete description of each algorithm and the eventual parameters that can be used to fine tune the detection can be found [here](https://scikit-learn.org/).

__Fit and evaluate different models on our THEMIS B labelled dataset using the function $\textit{train_evaluate}$. Save your best model using, which algorithm performed as the best ?__

In [ ]:
# This is the function you will use to train and evaluate your various models
from sklearn.model_selection import train_test_split

def train_evaluate(model, data, label):
    recalls = []
    precisions = []
    for i in range(3):
        print('Step ' + str(i), end='\r')
        X_train, X_test, y_train,  y_test = train_test_split(data, label)
        algo = model
        algo.fit(X_train, y_train)
        y_pred = algo.predict(X_test)
        recalls.append(recall_score(y_test, y_pred, average=None))
        precisions.append(precision_score(y_test, y_pred, average=None))
    print('Magnetosphere:')
    print('recall = '+ str(round(np.mean([x[0] for x in recalls]), 3)) +' +- '+str(round(np.std([x[1] for x in recalls]), 3)))

    print('precision = '+ str(round(np.mean([x[0] for x in precisions]), 3)) +' +- '+str(round(np.std([x[1] for x in precisions]), 3)))
    print('Magnetosheath:')
    print('recall = '+ str(round(np.mean([x[1] for x in recalls]), 3)) +' +- '+str(round(np.std([x[1] for x in recalls]), 3)))
    print('precision = '+ str(round(np.mean([x[1] for x in precisions]), 3)) +' +- '+str(round(np.std([x[1] for x in precisions]), 3)))
    
    print('Solar Wind:')
    print('recall = '+ str(round(np.mean([x[2] for x in recalls]), 3)) +' +- '+str(round(np.std([x[1] for x in recalls]), 3)))
    print('precision = '+ str(round(np.mean([x[2] for x in precisions]), 3)) +' +- '+str(round(np.std([x[1] for x in precisions]), 3))) 
    
    return recalls, precisions, model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier()

recalls, precisions, model = train_evaluate(model, X, y)

You can eventually plot the prediction of your model on one of the intervals you have used to visualize your data

In [ ]:
start = datetime.datetime(2008,5,12, 8)
end =  datetime.datetime(2008,5,12, 20)
event = evt.Event(start, end)


plotData(event, data_themis_B, spectro=spectro_themis_B, delta=datetime.timedelta(hours=0), pred = pd.Series(index=data_themis_B[event.begin:event.end].index, data = model.predict(data_themis_B[event.begin:event.end])))

## Construction of a Magnetopause Crossing list

Now we have a decent classification model, we can use it to elaborate our own Magnetopause Crossings list, assuming our model works well, we will run it on the data provided by both THEMIS B and THEMIS C.
For this,; we will define magnetopause crossing as 1hr intervals that contain as much points labelled as magnetosphere as points labelled as magnetosheath (Naturally, the same thing could be done with the bow shock).

__Using your best model, elaborate MP crossing lists for both Themis B and Themis C__

In [ ]:
#Load the data of Themis C
data_themis_C = pd.read_pickle('data_themisC.pkl')


In [ ]:
def make_crossing(data, model):
    crossings = []
    pred = model.predict(data)   #predict
    pred = pd.Series(index = data.index, data = medfilt(pred, 3))
    
    # Find the 1hr interval that contain as much MSH as MSP
    is_crossing = pred[(pred.rolling(60, center=True).sum() == 30) & (pred.rolling(60, center=True, min_periods=1).min()==0) & (pred.rolling(60, center=True, min_periods=1).max()==1)].index
    if len(is_crossing)>0:
        crossings += [evt.Event(center-datetime.timedelta(minutes=30), center+datetime.timedelta(minutes=30)) for center in is_crossing]  
    indices_to_keep= []
    
    # Separate overlapping crossings
    for i, x in enumerate(crossings[1:]):
        if evt.overlap(crossings[i], crossings[i-1])<datetime.timedelta(minutes=1):
            indices_to_keep.append(i)
    crossings = [crossings[i] for i in range(0, len(crossings)) if i in indices_to_keep]
    
    
    # remove the orbit moments where the spacecraft is in the magnetotail and the moment the spacecraft is around the earth dipole
    crossings = [x for x in crossings if (x.end<datetime.datetime(2007,12,15)) or (x.begin>datetime.datetime(2008,4,15))]
    crossings = [x for x in crossings if (x.end<datetime.datetime(2008,12,15)) or (x.begin>datetime.datetime(2009,4,15))]
    crossings = [x for x in crossings if data.Bx[x.begin:x.end].abs().max()<60]
    
    return crossings

In [ ]:
crossings_themisB = make_crossing(data_themis_B, model)

In [ ]:
crossings_themisC = make_crossing(data_themis_C, model)

All of these crossings can be represented together in the (X-Y), (X-Z) and (Y-Z) planes, the following cell provides a 2D histogram of the position of the crossings you have built. Make sure you haveve a decent coverage of the magnetopause :)

In [ ]:
def plot_boundaries():
    # Plot Standard BS and MP models projected in the three (X-Y), (X-Z) and (Y-Z) planes
    fig, axarr = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

    #phi = np.arange(0, np.pi+0.01, 0.01)
    theta = np.arange(0, np.pi+0.01, 0.01)
    phi = 0
    #theta = np.pi/2
    r = models.BS_Jerab05(phi, theta,1, 8 ,0.01)
    x = r*np.cos(theta)
    y = r*np.sin(theta)*np.cos(phi)
    z = r*np.sin(theta)*np.sin(phi)
    axarr[0].plot(x, y, color='k')

    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[0].plot(x, y, color='k', ls='dashed')


    theta = np.arange(0, np.pi+0.01, 0.01)
    phi = np.pi
    r = models.BS_Jerab05(phi, theta,1, 8 ,0.01)
    x = r*np.cos(theta)
    y = r*np.sin(theta)*np.cos(phi)
    z = r*np.sin(theta)*np.sin(phi)
    axarr[0].plot(x, y, color='k')

    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[0].plot(x, y, color='k', ls='dashed')

    #phi = np.arange(0, np.pi+0.01, 0.01)
    theta = np.arange(0, np.pi+0.01, 0.01)
    phi = np.pi/2
    #theta = np.pi/2
    r = models.BS_Jerab05(phi, theta,1, 8 ,0.01)
    x = r*np.cos(theta)
    y = r*np.sin(theta)*np.cos(phi)
    z = r*np.sin(theta)*np.sin(phi)
    axarr[1].plot(x, z, color='k')

    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[1].plot(x, z, color='k', ls='dashed')
    
    
    theta = np.arange(0, np.pi+0.01, 0.01)
    phi = -np.pi/2
    r = models.BS_Jerab05(phi, theta,1, 8 ,0.01)
    x = r*np.cos(theta)
    y = r*np.sin(theta)*np.cos(phi)
    z = r*np.sin(theta)*np.sin(phi)
    axarr[1].plot(x, z, color='k')

    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[1].plot(x, z, color='k', ls='dashed')

    phi = np.arange(0, np.pi+0.01, 0.01)
    theta = -np.pi/2
    r = models.BS_Jerab05(phi, theta,1, 8 ,0.01)
    x = r*np.cos(theta)
    y = r*np.sin(theta)*np.cos(phi)
    z = r*np.sin(theta)*np.sin(phi)
    axarr[2].plot(y, z, color='k')    

    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[2].plot(y, z, color='k', ls='dashed')
    
    
    phi = np.arange(0, np.pi+0.01, 0.01)
    theta = np.pi/2
    r = models.BS_Jerab05(phi, theta,1, 8 ,0.01)
    x = r*np.cos(theta)
    y = r*np.sin(theta)*np.cos(phi)
    z = r*np.sin(theta)*np.sin(phi)
    axarr[2].plot(y, z, color='k')   
    
    
    
    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[2].plot(y, z, color='k', ls='dashed')
    
    for ax in axarr:
        ax.set_xlim(-40, 40)
        ax.set_ylim(-40, 40)
    axarr[0].set_xlabel('X (Re)')
    axarr[0].set_ylabel('Y (Re)')
    
    axarr[1].set_xlabel('X (Re)')
    axarr[1].set_ylabel('Z (Re)')
    
    axarr[2].set_xlabel('Y (Re)')
    axarr[2].set_ylabel('Z (Re)')
    return fig, axarr

In [ ]:
#load the position of the Themis B and Themis C spacecrafts
themisB_pos = pd.read_pickle('themisB_pos.pkl')
themisC_pos = pd.read_pickle('themisC_pos.pkl')


In [ ]:
# plot the position of each crossings
fig, axarr = plot_boundaries()

posX = []
posY = []
posZ = []



for x in crossings_themisB:
    posX.append(themisB_pos['X'][x.begin:x.end].mean())
    posY.append(themisB_pos['Y'][x.begin:x.end].mean())
    posZ.append(themisB_pos['Z'][x.begin:x.end].mean())


for x in crossings_themisC:
    posX.append(themisC_pos['X'][x.begin:x.end].mean())
    posY.append(themisC_pos['Y'][x.begin:x.end].mean())
    posZ.append(themisC_pos['Z'][x.begin:x.end].mean())


axarr[0].hist2d(posX, posY, bins=np.arange(-20, 20, 1), cmap='plasma', norm=LogNorm())
axarr[1].hist2d(posX, posZ, bins=np.arange(-20, 20, 1), cmap='plasma', norm=LogNorm(), cmin=1)
seum = axarr[2].hist2d(posY, posZ, bins=np.arange(-30, 30, 1), cmap='plasma', norm=LogNorm())

plt.colorbar(seum[3], label='number of crossings')

# Construction of a data-driven magnetopause model

Once we have consequent magnetopause crossing list, one could think of linking them to the solar wind parameters at the moment the crossing happened. Once again, we could here use machine learning for regression purpose to try to fit the position of the magnetopause as a function of these upstream conditions. 
Nevertheless, the crossings we have been detecting for THEMIS only cover the near-equator part of the magnetopause and consequently do not provide a coverage significant enough for our study.

To cope with it, we provide you a complete dataset of magnetopause crossings, collected from the 5 THEMIS spacecrafts, Cluster 1 and 3 and the Double star TC1 spacecraft, and their associated SW conditions.

Here, we will try several typical supervised machine learning algorithms to figure out how each of them deals with regression problems in the specific case of the magnetopause position. 

Feel free to try the various existing algorithms and eventually fine tune them to increase the quality of your fit performances:

- Decision Tree
- Random Forest
- Gradient Boosting
- Support Vector Machine

The complete description of each algorithm and the eventual parameters that can be used to fine tune the detection can be found here.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def plot_MP_model():
    # Plot Standard BS and MP models projected in the three (X-Y), (X-Z) and (Y-Z) planes
    fig, axarr = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

    theta = np.arange(0,np.pi+0.01, 0.01)
    phi = 0

    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[0].plot(x, y, color='k', ls='dashed')


    theta = np.arange(0, np.pi+0.01, 0.01)
    phi = np.pi

    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[0].plot(x, y, color='k', ls='dashed')

    theta = np.arange(0, np.pi+0.01, 0.01)
    phi = np.pi/2


    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[1].plot(x, z, color='k', ls='dashed')
    
    
    theta = np.arange(0, np.pi+0.01, 0.01)
    phi = -np.pi/2

    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[1].plot(x, z, color='k', ls='dashed')

    phi = np.arange(0, np.pi+0.01, 0.01)
    theta = -np.pi/2   

    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[2].plot(y, z, color='k', ls='dashed')
    
  
    phi = np.arange(0, np.pi+0.01, 0.01)
    theta = np.pi/2  
    
    
    r, Q = models.MP_Lin2010(phi, theta,2, 0.01, 0, 0)
    x = (r+Q)*np.cos(theta)
    y = (r+Q)*np.sin(theta)*np.cos(phi)
    z = (r+Q)*np.sin(theta)*np.sin(phi)
    axarr[2].plot(y, z, color='k', ls='dashed')
    
    for ax in axarr:
        ax.set_xlim(-20, 20)
        ax.set_ylim(-20, 20)
    axarr[0].set_xlabel('X (Re)')
    axarr[0].set_ylabel('Y (Re)')
    
    axarr[1].set_xlabel('X (Re)')
    axarr[1].set_ylabel('Z (Re)')
    
    axarr[2].set_xlabel('Y (Re)')
    axarr[2].set_ylabel('Z (Re)')
    return fig, axarr

In [ ]:
dataset_for_regression = pd.read_csv('dataset_for_regression.csv', index_col=0)

Each line of the dataset here corresponds to a given crossing, its position in both GSE coordinate, $X, Y, Z$, and spherical coordinates, $R, \theta, \phi$, and the associated magnetic field $B_{z}$, Alfven Mach number $M_{a}$ and dynamic pressure $P_{dyn}$

In [ ]:
dataset_for_regression.head()

In [ ]:
#Current coverage of the MP by our dataset

fig, axarr = plot_MP_model()

axarr[0].scatter(dataset_for_regression.R*np.cos(dataset_for_regression.theta),
                 dataset_for_regression.R*np.sin(dataset_for_regression.theta)*np.cos(dataset_for_regression.phi), s=1)

axarr[1].scatter(dataset_for_regression.R*np.cos(dataset_for_regression.theta),
                 dataset_for_regression.R*np.sin(dataset_for_regression.theta)*np.sin(dataset_for_regression.phi), s=1)

axarr[2].scatter(dataset_for_regression.R*np.sin(dataset_for_regression.theta)*np.cos(dataset_for_regression.phi),
                 dataset_for_regression.R*np.sin(dataset_for_regression.theta)*np.sin(dataset_for_regression.phi), s=1)


The objective here is to estimate the Radius R of the bow shock as a function of the two other coordinates $\theta$ and $\phi$ and the eventual solar wind conditions:

$R = f(\theta, \phi, B_{z}, M_{a}, P_{dyn})$

The dataset will then consist of these parameter while $R$ will be the value to estimate

__Fit and evaluate different models on our THEMIS B labelled dataset and look at how the different algorithms deal with the regression of the MP position__

At first, you can eventually try a first fit independent from the solar wind conditions

In [ ]:
X = dataset_for_regression[['theta', 'phi']]

In [ ]:
Radius = dataset_for_regression.R

To enhance our dataset, we can eventually assume the magnetopause is a couple of symmetries:
$R(X, -Y, Z) = R(X, Y, -Z) = R(X, Y, Z)  $

This is true as long as we do not take into consideration the eventual seasonal influence on the magnetopause location, which will be the case here.


In [ ]:
X_sym = X.copy()
X_sym.phi *=-1
X_sym.phi += np.pi

X_sym2 = pd.concat([X, X_sym])
X_sym2.phi*= -1
X_sym2.phi += 2*np.pi

Now let's get into serious stuffs

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pd.concat([X, X_sym, X_sym2]), pd.concat([Radius, Radius, Radius, Radius]))

model = GradientBoostingRegressor(verbose=1, learning_rate=0.05, min_samples_split=10)
model.fit(X_train, y_train)

In [ ]:
#projection plot
fig, axarr = plot_MP_model()



theta = np.arange(0, 2*np.pi/3+0.01, 0.01)
phi = np.zeros(len(theta))

r = model.predict(pd.DataFrame(np.concatenate((theta, phi)).reshape(2, len(theta))).transpose())
r = np.around(r, 1)
x = r*np.cos(theta)
y = r*np.sin(theta)*np.cos(phi)
z = r*np.sin(theta)*np.sin(phi)
axarr[0].plot(x, y, color='k')


theta = np.arange(0, 2*np.pi/3+0.01, 0.01)
phi = np.pi*np.ones(len(theta))

r = model.predict(pd.DataFrame(np.concatenate((theta, phi)).reshape(2, len(theta))).transpose())
r = np.around(r, 1)
x = r*np.cos(theta)
y = r*np.sin(theta)*np.cos(phi)
z = r*np.sin(theta)*np.sin(phi)
axarr[0].plot(x, y, color='k')



theta = np.arange(0, 2*np.pi/3+0.01, 0.01)
phi = np.pi*np.ones(len(theta))*0.5


r = model.predict(pd.DataFrame(np.concatenate((theta, phi)).reshape(2, len(theta))).transpose())
r = np.around(r, 1)
x = r*np.cos(theta)
y = r*np.sin(theta)*np.cos(phi)
z = r*np.sin(theta)*np.sin(phi)
axarr[1].plot(x, z, color='k')


theta = np.arange(0, 2*np.pi/3+0.01, 0.01)
phi = -np.pi*np.ones(len(theta))*0.5

r = model.predict(pd.DataFrame(np.concatenate((theta, phi)).reshape(2, len(theta))).transpose())
r = np.around(r, 1)
x = r*np.cos(theta)
y = r*np.sin(theta)*np.cos(phi)
z = r*np.sin(theta)*np.sin(phi)
axarr[1].plot(x, z, color='k')





phi = np.arange(0, 2*np.pi+0.01, 0.01)
theta = np.pi*np.ones(len(phi))*0.5 


r = model.predict(pd.DataFrame(np.concatenate((theta, phi)).reshape(2, len(phi))).transpose())
r = np.around(r, 1)
x = r*np.cos(theta)
y = r*np.sin(theta)*np.cos(phi)
z = r*np.sin(theta)*np.sin(phi)
axarr[2].plot(y, z, color='k')

